In [0]:
/* 

Filters:
FTE table
*Only Base records from Entity Hierarchy for US and Canada
*CostType = 'COS DIRECT'
*Breakdown2Id in ('TKE101','TKE102','TKE1031','TKE1032')
*fte <> 0 (Value)

Result:
For LeadershipScorecard; Auto_FctData

*/
WITH onlybaseentities as (
  SELECT
    child
  FROM
    `na-bu-cdm-dev`.`staging_usdwh_tkefinance`.`epmparentchildhierarchy`
  WHERE
    `IsBaseRecord` = 1
    AND (
      `adjacencypath` LIKE
        'Entity|OEP_Total Entity|Total_Legal|Vertical_US_Newco_Inc_and_Subs|tk_Elevator_USA_Holding_CON|tk_Elevator_Americas_Corp_and_Subs|tk_Elevator_Corp_and_Subs|tk_Elevator_Corporate|901745_01_SAP|tk_Elevator_Corp_USField|%'
      OR `adjacencypath` LIKE
        'Entity|OEP_Total Entity|Total_Legal|thyssenkrupp_Canada_Holding_and_Subs|tk_Northern_Elevator_Corp_and_Subs|tk_Elevator_Canada_Ltd|%'
    )
),
branchnames as (
  SELECT
    backendname,
    alias
  FROM
    `na-bu-cdm-dev`.`staging_usdwh_tkefinance`.`epmdimensionmetadata`
),
alldirectpriortogrouping as (
  SELECT
    'Direct ' as jobtype,
    CASE
      WHEN jobcodecategory = 'Mechanic' THEN 'Mechanic '
      When jobcodecategory is null THEN 'Mechanic '
      WHEN jobcodecategory = 'US SALES REPS' THEN 'Sales '
      WHEN jobcodecategory = 'Mechanic Temp' THEN 'Mechanic Temp '
      WHEN jobcodecategory = 'MIC' THEN 'MIC '
      WHEN jobcodecategory = 'Probationary' THEN 'Apprentice '
      WHEN jobcodecategory = '1st Year' THEN '1st Year '
      WHEN jobcodecategory = '2nd Year' THEN '2nd Year '
      WHEN jobcodecategory = '3rd Year' THEN '3rd Year '
      WHEN jobcodecategory = '4th Year' THEN '4th Year '
      WHEN jobcodecategory = 'Adjuster' THEN 'Adjuster '
      --WHEN jobcodecategory IS NULL THEN 'HC '
      ELSE
        jobcodecategory
    END AS jobcategory,
    CASE
      WHEN Breakdown2Id = 'TKE101' THEN 'NI'
      WHEN Breakdown2Id = 'TKE102' THEN 'MOD'
      WHEN Breakdown2Id = 'TKE1031' THEN 'SERV'
      WHEN Breakdown2Id = 'TKE1032' THEN 'REPAIR'
      ELSE 'Breakdown2Id not classifed'
    END AS LOB,
    --BreakDownTwo,
    country,
    branch.alias as entity,
    TRUNC(TO_DATE(CAST(dateint AS STRING), 'yyyyMMdd'), 'MONTH') as `Calendar Date`,
    --to_date(CAST(dateint as string), 'yyyyMMdd') as `Calendar Date`,
    fte
  FROM
    `na-cdp-bu-hr-dev`.curated.fte as fte
      JOIN onlybaseentities as child
        on child.child = fte.epmbranch
      LEFT JOIN branchnames as branch
        on branch.backendname = fte.epmbranch
  WHERE
    CostType = 'COS DIRECT'
    and Breakdown2Id in ('TKE101', 'TKE102', 'TKE1031', 'TKE1032')
    --and IsOverScale <> 'Y'
    and fte <> 0
  GROUP BY
    ALL
),
directjobtype_jobcategory_breakdown2id as (
  SELECT
    concat(jobtype, jobcategory, LOB) as account,
    'Actuals' as Scenario,
    `Calendar Date`,
    entity as Entity,
    ROUND(sum(fte), 3) as Value
  FROM
    alldirectpriortogrouping
  GROUP BY
    ALL
),
totalBuNa as (
  SELECT
    concat(jobtype, jobcategory, LOB) as account,
    'Actuals' as Scenario,
    `Calendar Date`,
    'BU North America' as Entity,
    --entity as Entity,
    ROUND(sum(fte), 3) as Value
  FROM
    alldirectpriortogrouping
  GROUP BY
    ALL
)
SELECT * FROM
(SELECT
  *
FROM
  directjobtype_jobcategory_breakdown2id
UNION ALL
SELECT
  *
FROM
  totalbuna) where `Calendar Date` = make_date(2025,1,1) and entity = 'BU North America' and account like '% NI'